In [16]:
from scripts.google_api import get_form
from scripts.data_processing import clean_sheet

# Get the form responses
get_form()
clean_sheet()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=997149058913-gs5122uc033sk42ohidnjs9pvfdhjsik.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59979%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=rknyODBdaxJNKwE0Q5OhmskLute2vs&access_type=offline


,Timestamp_1,Column_No,Check-in date,Check-out date,City of birth,City of residence,Country of issue,Country of residence,Date of birth,First name,Last name,Nationality,Passport (or ID) number
0,3/6/2024 13:37:23,3,22-02-2019,21-02-2029,Cahors,Toulouse,France,France,20-02-1997,Martin,Silva,French,19AP24609
1,9/27/2023 22:44:34,1,14-09-2023,29-09-2023,dsd,sc,Australia,Australia,01-09-2023,Test,,Angolan,adad
2,3/6/2024 13:37:23,2,21-09-2023,20-09-2033,Toulouse,Toulouse,France,France,11-12-2002,Lucie,Hazan,French,23Ik81395
3,9/29/2023 12:37:34,3,04-10-2023,10-10-2023,Poland Gliwice,Warsaw,Poland,Poland,04-10-1947,Boguszewska-Baczkowska,,Polish,CFb 269735
4,9/29/2023 12:37:34,2,04-10-2023,10-10-2023,Poland Kielce,Warsaw,Poland,Poland,16-06-1947,Baczkowski,,Polish,CFi 194276
5,10/4/2023 23:31:51,2,11-10-2023,15-10-2023,SELANGOR,London,United Kingdom,United Kingdom,28-07-1989,Chee Huan,Lee,Malaysian,A55269113
6,10/4/2023 23:31:51,3,11-10-2023,15-10-2023,Vasteras,London,Sweden,United Kingdom,13-10-1988,Lars Dennis,Ramstedt,Swedish,878789091
7,10/6/2023 3:20:18,2,15-10-2023,17-10-2023,"St. Charles, Missouri","Jacksonville, Florida",United States,United States,25-06-1975,Amber,Burgbacher,American,A30117927
8,10/6/2023 3:20:18,3,15-10-2023,17-10-2023,"Memphis, Tennessee","Jacksonville, Florida",United States,United States,07-12-1977,Chad,Burgbacher,American,664791199
9,9/30/2023 7:32:43,2,19-10-2023,22-10-2023,Queens,Alexandria,United States,United States,13-03-1998,Zion,Khanna,American,T69671963


In [21]:
# Get the form responses
get_form()
clean_df = clean_sheet()

def filter_df_on_name(clean_df, first_name=None, last_name=None):

    # Filter the DataFrame based on combinations of first and last name
    if first_name and last_name:
        filtered_df = clean_df[(clean_df['First name'] == first_name) & (clean_df['Last name'] == last_name)]
    elif first_name:
        filtered_df = clean_df[clean_df['First name'] == first_name]
    elif last_name:
        filtered_df = clean_df[clean_df['Last name'] == last_name]
    else:
        filtered_df = clean_df

    return filtered_df

# filter_df_on_name(clean_df, first_name='Lucie', last_name=None)
# filter_df_on_name(clean_df, first_name=None, last_name='Hazan')
# filter_df_on_name(clean_df, first_name='Lucie', last_name='Hazan')
filter_df_on_name(clean_df, first_name='Lucie', last_name='Hazan')

,Timestamp_1,Column_No,Check-in date,Check-out date,City of birth,City of residence,Country of issue,Country of residence,Date of birth,First name,Last name,Nationality,Passport (or ID) number
2,3/6/2024 13:37:23,2,21-09-2023,20-09-2033,Toulouse,Toulouse,France,France,11-12-2002,Lucie,Hazan,French,23Ik81395


In [23]:
string = ''

if not string:
    print('String is not empty')

String is not empty


In [ ]:
import json
from scripts.aws import *

key = 'records.json'
obj = download_object(bucket_name, key)
records = json.loads(obj)

records

In [ ]:
from scripts.data_processing import *
import json

records = write_new_records_to_json(None)

json.dumps(records)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

def test_duckduckgo_search():
    # Set up Chrome options for headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Ensure GUI is not displayed
    chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
    chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

    # Initialize WebDriver with options
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Navigate to DuckDuckGo
        driver.get("https://www.duckduckgo.com")
        assert "DuckDuckGo" in driver.title

        # Find the search box using its name attribute value
        search_box = driver.find_element("name", "q")
        search_box.clear()

        # Perform a search operation
        search_term = "selenium headless browser testing"
        search_box.send_keys(search_term)
        search_box.send_keys(Keys.RETURN)  # Simulate pressing the Enter key

        time.sleep(2)  # Wait a bit for the search results to load

        # Verify that the search operation was successful
        assert search_term in driver.title

        print("Test passed: Search operation was successful.")
    except AssertionError as e:
        print(f"Test failed: {e}")
    finally:
        driver.quit()

test_duckduckgo_search()


In [2]:
from scripts.data_processing import *
from scripts.google_api import *

checkin_date = '29-03-2024'
# Get form responses
# sheet_path = get_form()
clean_df = clean_sheet()
filtered_df = filter_df_on_checkin_date(clean_df, checkin_date)

filtered_df

,Timestamp_1,Column_No,Check-in date,Check-out date,City of birth,City of residence,Country of issue,Country of residence,Date of birth,First name,Last name,Nationality,Passport (or ID) number
28,3/28/2024 7:09:06,2,29-03-2024,01-04-2024,Pelhřimov,Praha,Portugal,Czech Republic,16-04-1985,Olga,Kučerová,Czech,47515859
29,3/28/2024 7:09:06,3,29-03-2024,01-04-2024,Rakovník,Praha,Portugal,Czech Republic,04-08-1975,Tomáš,Baldýnský,Czech,207867301


In [17]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from scripts import google_api, web_automation, data_processing
from dotenv import load_dotenv

# Load environment variables from a .env file located in the same directory as this script
load_dotenv()
uh_input_ = os.getenv('SEF_UH')
estabelecimento_input_ = os.getenv('SEF_ESTABELECIMENTO')
chave_activacao_input_ = os.getenv('SEF_CHAVE')
pdf_nif = os.getenv('PDF_NIF')
pdf_senha = os.getenv('PDF_SENHA')

# Extract the mapping files
countries_mapping = os.path.join(os.getcwd(), 'parameters', 'countries_mapping.json')
nationalities_mapping = os.path.join(os.getcwd(), 'parameters', 'nationalities_mapping.json')

def callback(msg):
    print(msg)

def fill_in_sef_form(df, callback):
    try:
        with open(countries_mapping) as f:
            countries = json.load(f)

        with open(nationalities_mapping) as f:
            nationalities = json.load(f)

        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        web = webdriver.Chrome(options=chrome_options)

        callback("Opening the SEF website...")
        url = 'https://siba.sef.pt/s/FB.aspx?ReturnUrl=%2fs%2fbal%2fLotesEnvio.aspx'
        web.get(url)
        time.sleep(2)

        callback("Filling in the establishment details...")
        uh_input = web.find_element('id', 'Conteudo_txtUH')
        estabelecimento_input = web.find_element('id', 'Conteudo_txtEstabelecimento')
        chave_activacao_input = web.find_element('id', 'Conteudo_txtChaveActivacao')

        uh_input.send_keys(uh_input_)
        estabelecimento_input.send_keys(estabelecimento_input_)
        chave_activacao_input.send_keys(chave_activacao_input_)

        # submit_button = web.find_element('id', 'Conteudo_btnConfirmar')
        # submit_button.click()

        # Example for the submit button
        # submit_button = WebDriverWait(web, 10).until(
        #     EC.element_to_be_clickable((By.ID, 'Conteudo_btnConfirmar'))
        # )
        # submit_button.click()

        # submit_button = web.find_element('id', 'Conteudo_btnConfirmar')
        # ActionChains(web).move_to_element(submit_button).perform()
        # submit_button.click()

        submit_button = web.find_element('id', 'Conteudo_btnConfirmar')
        web.execute_script("arguments[0].click();", submit_button)

        time.sleep(2)

        callback("Creating a new list...")
        new_list = web.find_element('id', 'Conteudo_btnNovaLista')
        new_list.click()
        time.sleep(2)

        callback("Editing the list...")
        edit_list = web.find_element('id', 'Conteudo_dg_btnSelect_0')
        edit_list.click()
        time.sleep(2)

        try:
            callback("Trying to add new bulletin in case list already existed...")
            add_bulletin = web.find_element('id', 'Conteudo_btnNovaBAL')
            # add_bulletin.click()
            web.execute_script("arguments[0].click();", add_bulletin)
            callback("Added new bulleting...")
            time.sleep(2)
        except Exception as e:
            callback(f"Skipped adding new bulleting. Error: {e}")
            time.sleep(1)

        callback("Preparing to fill in guest details...")
        for idx, row in df.iterrows():
            name_ = web.find_element('id', 'Conteudo_txtNome')
            dob_ = web.find_element('id', 'Conteudo_txtDataNascimento')
            nationality_ = web.find_element('id', 'Conteudo_lstNacionalidade')
            country_of_res_ = web.find_element('id', 'Conteudo_lstPaisResidencia')
            passport_ = web.find_element('id', 'Conteudo_txtNumPassaporteBI')
            country_of_issue_ = web.find_element('id', 'Conteudo_lstPaisEmissor')
            checkin_date_ = web.find_element('id', 'Conteudo_txtDataEntrada')
            checkout_date_ = web.find_element('id', 'Conteudo_txtDataSaida')

            name = row['First name'] + ' ' + row['Last name']
            dob = row['Date of birth']
            nationality = countries[nationalities[row['Nationality']]]
            country_of_res = countries[row['Country of residence']]
            passport = row['Passport (or ID) number']
            country_of_issue = countries[row['Country of issue']]
            checkin_date = row['Check-in date']
            checkout_date = row['Check-out date']
            print("before sending keys")
            name_.send_keys(name)
            print("name done")
            dob_.send_keys(dob)
            print("dob_ done")
            nationality_.send_keys(nationality)
            # web.execute_script("arguments[0].value = arguments[1];", nationality_, nationality)
            print("nationality_ done")
            country_of_res_.send_keys(country_of_res)
            print("country_of_res_ done")
            passport_.send_keys(passport)
            print("passport_ done")
            country_of_issue_.send_keys(country_of_issue)
            print("country_of_issue_ done")
            checkin_date_.send_keys(checkin_date)
            print("checkin_date_ done")
            checkout_date_.send_keys(checkout_date)
            print("checkout done")

            # Using JavaScript to set the values
            # web.execute_script("arguments[0].value = arguments[1];", name_, name)
            # print("name done")
            # web.execute_script("arguments[0].value = arguments[1];", dob_, dob)
            # print("dob_ done")
            # web.execute_script("arguments[0].value = arguments[1];", nationality_, nationality)
            # print("nationality_ done")
            # web.execute_script("arguments[0].value = arguments[1];", country_of_res_, country_of_res)
            # print("country_of_res_ done")
            # web.execute_script("arguments[0].value = arguments[1];", passport_, passport)
            # print("passport_ done")
            # web.execute_script("arguments[0].value = arguments[1];", country_of_issue_, country_of_issue)
            # print("country_of_issue_ done")
            # web.execute_script("arguments[0].value = arguments[1];", checkin_date_, checkin_date)
            # print("checkin_date_ done")
            # web.execute_script("arguments[0].value = arguments[1];", checkout_date_, checkout_date)
            # print("checkout done")

            print("after sending keys")
            save = web.find_element('id', 'Conteudo_btnActualizarBAL')
            save.click()
            # web.execute_script("arguments[0].click();", save)

            callback(f"Details for {name} saved. Proceeding to the next guest...")
            time.sleep(3)

        callback("Finalizing and sending the list...")
        send = web.find_element('id', 'Conteudo_btnEnviarLista')
        # web.execute_script("arguments[0].click();", send)
        send.click()
        time.sleep(3)

        callback("SEF registration completed successfully.")

    except Exception as e:
        callback(f"Error during SEF registration: {e}")

    finally:
        web.quit()

####################################################################################################
        
def fill_in_invoice(callback, filtered_df, amount, date=None, invoice_nif=None):

    if filtered_df.empty:
        print('Data provided is empty.')
        message = 'Data provided is empty.'
        return message
    
    # Extract info from df
    row = filtered_df.iloc[0]
    if not date:
        date = pd.to_datetime(row['Check-out date'], dayfirst=True).strftime('%Y-%m-%d')
    else:
        date = date.strftime('%Y-%m-%d')

    try:

        with open(countries_mapping) as f:
            countries = json.load(f)

        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        web = webdriver.Chrome(options=chrome_options)

        callback("Opening the IRS website...")
        url = r'https://irs.portaldasfinancas.gov.pt/recibos/portal/emitir/emitirDocumentos'
        web.get(url)

        time.sleep(2)

        callback("Filling in the login details...")
        # Find the label element by its text content (assuming "NIF" is the text)
        label_element = web.find_element(By.XPATH, "//label[span='NIF']")

        # Click the label element to perform the action
        label_element.click()
        time.sleep(1)

        # Find and fill in the input fields
        nif = web.find_element('id', 'username')
        senha = web.find_element('id', 'password-nif')

        nif.send_keys(pdf_nif)
        senha.send_keys(pdf_senha)

        # Locate and click the submit button
        submit_button = web.find_element('id', 'sbmtLogin')
        submit_button.click()
        time.sleep(2)

        callback("Logged in successfully.")
        time.sleep(1)

        callback("Filling in the date and type of invoice...")
        # Input date and type of invoice
        date_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-documentos-app-v2/emitir-documentos-form-v2/div[1]/div[2]/div/div/div/div[3]/div/div[1]/lf-date/div/div[1]/input')
        date_.send_keys(date)

        type_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-documentos-app-v2/emitir-documentos-form-v2/div[1]/div[2]/div/div/div/div[3]/div/div[2]/lf-dropdown/div/select')
        type_.send_keys('Fatura-Recibo')
        
        # Scroll into view (if needed)
        emitir_button = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-documentos-app-v2/emitir-documentos-form-v2/div[1]/div[2]/div/div/div/div[3]/div/div[3]/button')
        web.execute_script("arguments[0].scrollIntoView(true);", emitir_button)
        time.sleep(1)  # Just to be safe, give it a moment

        # Click using JavaScript
        web.execute_script("arguments[0].click();", emitir_button)    
        time.sleep(2)

        callback("Filling in the invoice details...")
        # Get all fields
        country_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-adquirente/div/div[2]/div[1]/div[1]/lf-dropdown/div/select')
        name_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-adquirente/div/div[2]/div[2]/div/lf-text/div/input')
        payment_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[1]/pf-radio/div/div[1]/label/input')
        description_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[2]/div/textarea')
        iva_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[6]/div/div[1]/lf-dropdown/div/select')
        incidencia_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[7]/div[1]/div/lf-dropdown/div/select')
        amount_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[4]/div/div/div/input')
        selo_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[8]/div/div/div/input')

        # Write in fields
        country = countries[row['Country of residence']]
        country_.send_keys(country)

        if country.lower() == 'portugal':
            nif_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-adquirente/div/div[2]/div[1]/div[3]/lf-nif/div/input')
            nif_.send_keys(invoice_nif)
        else:
            passport_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-adquirente/div/div[2]/div[1]/div[2]/lf-text/div/input')
            passport = row['Passport (or ID) number']
            passport_.send_keys(passport)

        name = row['First name'] + ' ' + row['Last name']
        name_.send_keys(name)

        payment_.click()

        checkin_date = pd.to_datetime(row['Check-in date'], dayfirst=True).strftime('%d/%m/%Y')
        checkout_date = pd.to_datetime(row['Check-out date'], dayfirst=True).strftime('%d/%m/%Y')
        al_number = '138454/AL'
        address = 'RUA DE MARVILA N 54 R/C E 1950-199 LISBOA'
        description = f'Prestação de serviços de alojamento mobilado para turistas, da data {checkin_date} a {checkout_date}, no AL {al_number}, sito na morada: {address}'
        description_.send_keys(description)

        iva = 'Continente - 6% [taxa reduzida atual]'
        iva_.send_keys(iva)

        incidencia = 'Sem retenção - Art.101º, n.º1 do CIRS'
        incidencia_.send_keys(incidencia)

        amount = float(amount)
        amount = round(amount/1.06, 2) # Remove the 6% VAT
        amount = str(amount)
        amount_.send_keys(amount)

        selo = '0'
        selo_.send_keys(selo)
        time.sleep(2)

        callback("Submitting the invoice (first button)...")
        first_emitir_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[1]/div[1]/div[1]/div[3]/button')
        # first_emitir_.click()
        time.sleep(2)

        callback("Submitting the invoice (second button)...")
        second_emitir_ = web.find_element(By.XPATH, '//*[@id="emitirModal"]/div/div/div[3]/button[2]')
        # second_emitir_.click()
        time.sleep(2)

        message = 'Invoice Completed'

        print(message)

    except Exception as e:

        print(f"Error: {e}")
        message = e
    
    return message

# Get the form responses
checkout_date = '01-04-2024'
clean_df = data_processing.clean_sheet()
filtered_df = data_processing.filter_df_on_checkout_date(clean_df, checkout_date)
amount = '393.33'

fill_in_invoice(callback, filtered_df, amount)

Opening the IRS website...
Filling in the login details...
Logged in successfully.
Filling in the date and type of invoice...
Filling in the invoice details...
Submitting the invoice (first button)...
Submitting the invoice (second button)...
Invoice Completed


'Invoice Completed'

In [ ]:
import os
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from scripts import google_api, web_automation, data_processing
from dotenv import load_dotenv

# Load environment variables from a .env file located in the same directory as this script
load_dotenv()
pdf_nif = os.getenv('PDF_NIF')
pdf_senha = os.getenv('PDF_SENHA')

# Extract the mapping files
countries_mapping = os.path.join(os.getcwd(), 'parameters', 'countries_mapping.json')
nationalities_mapping = os.path.join(os.getcwd(), 'parameters', 'nationalities_mapping.json')

def fill_in_invoice(filtered_df, amount, date=None, invoice_nif=None):

    if filtered_df.empty:
        print('Data provided is empty.')
        message = 'Data provided is empty.'
        return message
    
    # Extract info from df
    row = filtered_df.iloc[0]
    if not date:
        date = pd.to_datetime(row['Check-out date'], dayfirst=True).strftime('%Y-%m-%d')
    else:
        date = date.strftime('%Y-%m-%d')

    try:

        with open(countries_mapping) as f:
            countries = json.load(f)

        web = webdriver.Chrome()

        url = r'https://irs.portaldasfinancas.gov.pt/recibos/portal/emitir/emitirDocumentos'
        web.get(url)

        time.sleep(2)

        # Find the label element by its text content (assuming "NIF" is the text)
        label_element = web.find_element(By.XPATH, "//label[span='NIF']")

        # Click the label element to perform the action
        label_element.click()

        time.sleep(1)

        # Find and fill in the input fields
        nif = web.find_element('id', 'username')
        senha = web.find_element('id', 'password-nif')

        nif.send_keys(pdf_nif)
        senha.send_keys(pdf_senha)

        # Locate and click the submit button
        submit_button = web.find_element('id', 'sbmtLogin')
        submit_button.click()

        time.sleep(2)

        # Input date and type of invoice
        date_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-documentos-app-v2/emitir-documentos-form-v2/div[1]/div[2]/div/div/div/div[3]/div/div[1]/lf-date/div/div[1]/input')
        date_.send_keys(date)

        type_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-documentos-app-v2/emitir-documentos-form-v2/div[1]/div[2]/div/div/div/div[3]/div/div[2]/lf-dropdown/div/select')
        type_.send_keys('Fatura-Recibo')

        emitir_button = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-documentos-app-v2/emitir-documentos-form-v2/div[1]/div[2]/div/div/div/div[3]/div/div[3]/button')
        emitir_button.click()        

        time.sleep(2)

        # Get all fields
        country_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-adquirente/div/div[2]/div[1]/div[1]/lf-dropdown/div/select')
        name_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-adquirente/div/div[2]/div[2]/div/lf-text/div/input')
        payment_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[1]/pf-radio/div/div[1]/label/input')
        description_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[2]/div/textarea')
        iva_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[6]/div/div[1]/lf-dropdown/div/select')
        incidencia_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[7]/div[1]/div/lf-dropdown/div/select')
        amount_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[4]/div/div/div/input')
        selo_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-transmissao/div/div[2]/div[8]/div/div/div/input')
        first_emitir_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[1]/div[1]/div[1]/div[3]/button')
        second_emitir_ = web.find_element(By.XPATH, '//*[@id="emitirModal"]/div/div/div[3]/button[2]')

        # Write in fields
        country = countries[row['Country of residence']]
        country_.send_keys(country)

        if country.lower() == 'portugal':
            nif_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-adquirente/div/div[2]/div[1]/div[3]/lf-nif/div/input')
            nif_.send_keys(invoice_nif)
        else:
            passport_ = web.find_element(By.XPATH, '//*[@id="main-content"]/div/div/emitir-app/emitir-form/div[1]/div[2]/div/dados-adquirente/div/div[2]/div[1]/div[2]/lf-text/div/input')
            passport = row['Passport (or ID) number']
            passport_.send_keys(passport)

        name = row['First name'] + ' ' + row['Last name']
        name_.send_keys(name)

        payment_.click()

        checkin_date = pd.to_datetime(row['Check-in date'], dayfirst=True).strftime('%d/%m/%Y')
        checkout_date = pd.to_datetime(row['Check-out date'], dayfirst=True).strftime('%d/%m/%Y')
        al_number = '138454/AL'
        address = 'RUA DE MARVILA N 54 R/C E 1950-199 LISBOA'
        description = f'Prestação de serviços de alojamento mobilado para turistas, da data {checkin_date} a {checkout_date}, no AL {al_number}, sito na morada: {address}'
        description_.send_keys(description)

        iva = 'Continente - 6% [taxa reduzida atual]'
        iva_.send_keys(iva)

        incidencia = 'Sem retenção - Art.101º, n.º1 do CIRS'
        incidencia_.send_keys(incidencia)

        amount = float(amount)
        amount = round(amount/1.06, 2) # Remove the 6% VAT
        amount = str(amount)
        amount_.send_keys(amount)

        selo = '0'
        selo_.send_keys(selo)

        time.sleep(2)

        # first_emitir_.click()

        # time.sleep(2)

        # second_emitir_.click()
        
        time.sleep(10)

        message = 'Invoice Completed'

        print(message)

    except Exception as e:

        print(f"Error: {e}")
        message = e
    
    return message

# Get the form responses
checkout_date = '01-04-2024'
clean_df = data_processing.clean_sheet()
filtered_df = data_processing.filter_df_on_checkout_date(clean_df, checkout_date)

amount = '393.33'

# fill_in_invoice(filtered_df, amount)